In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.autograd as grad

from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
import torchvision.datasets as dsets

In [2]:
# load data
mnist_train = dsets.MNIST(root='data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='data/', train=False, transform=transforms.ToTensor(), download=True)

In [8]:
print(mnist_train.data.shape, mnist_train.targets.shape)
print(mnist_test.data.shape, mnist_test.targets.shape)

torch.Size([60000, 28, 28]) torch.Size([60000])
torch.Size([10000, 28, 28]) torch.Size([10000])


In [9]:
# fix random seed
torch.manual_seed(2023)

# set parameters
num_epochs = 15
learning_rate = 0.001
batch_size = 100


In [10]:
data_loader = DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

In [11]:
# define model

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.num_classes = num_classes
        
        # 합성곱층, 활성화함수, 풀링층을 하나의 층으로 정의해버린다.
        self.layer_1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, padding = 1, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2) # 커널 사이즈, 스트라이드 = 2
        )
        self.layer_2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(7 * 7 * 64, num_classes, bias = True)
        nn.init.xavier_uniform_(self.fc.weight) # 전결합층 한정 가중치를 초기화 한다.
        
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
# define loss function and optimizer
model = CNN(10)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [13]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [14]:
# train
for epoch in range(num_epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        optimizer.zero_grad()
        y_pred = model(X)
        
        cost = loss_func(y_pred, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch : {:04d}, Cost : {:.9f}'.format(epoch + 1, avg_cost))
        

Epoch : 0001, Cost : 0.212866664
Epoch : 0002, Cost : 0.061608903
Epoch : 0003, Cost : 0.044519044
Epoch : 0004, Cost : 0.037997745
Epoch : 0005, Cost : 0.029879838
Epoch : 0006, Cost : 0.025223691
Epoch : 0007, Cost : 0.022161689
Epoch : 0008, Cost : 0.017780701
Epoch : 0009, Cost : 0.014811752
Epoch : 0010, Cost : 0.013788257
Epoch : 0011, Cost : 0.010522249
Epoch : 0012, Cost : 0.009566001
Epoch : 0013, Cost : 0.008549294
Epoch : 0014, Cost : 0.005482315
Epoch : 0015, Cost : 0.005549218


In [19]:
# test model
model.eval()

with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float()
    y_test = mnist_test.targets
    
    pred = model(X_test)
    correct_prediction = torch.argmax(pred, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.9865999817848206


In [18]:
mnist_test.targets

tensor([7, 2, 1,  ..., 4, 5, 6])